<a href="https://colab.research.google.com/github/Kewin043/AlfidoTech/blob/main/Tapex_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.8 MB/s eta 0:00:00


In [2]:
! pip install transformers

In [3]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [5]:
import pandas as pd
data = pd.read_csv("/content/10000 Sales Records.csv")

In [6]:
data = data.astype(str)

In [7]:
query = [
    "Which region has the highest total profit?",
    "Which country has the highest total profit?",
    "Which country generates the highest total revenue?"
]

In [8]:
from transformers import TapexTokenizer, BartForConditionalGeneration

# Tokenize the query and table
tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-large-finetuned-wtq")
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-large-finetuned-wtq")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/506k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/951 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/246 [00:00<?, ?B/s]

In [9]:

# Truncate the input to fit within the model's maximum sequence length
max_length = model.config.max_position_embeddings
encoding = tokenizer(table=data, query=query, return_tensors="pt", truncation=True, max_length=max_length)

# Generate the output
outputs = model.generate(**encoding)


In [10]:
def predict_with_model(model, tokenizer, encoding):
    # Generate the output
    outputs = model.generate(**encoding)
    # Decode the output
    predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return predictions
    # Generate and decode predictions
predictions = predict_with_model(model, tokenizer, encoding)
print(predictions)


[' europe', ' latvia', ' chad']


In [14]:
from transformers import TapexTokenizer, BartForConditionalGeneration

def predict_with_model(model, tokenizer, data, queries):
    # Initialize an empty list to store predictions
    predictions = []

    # Tokenize the query and table
    max_length = model.config.max_position_embeddings
    for query in queries:
        encoding = tokenizer(table=data, query=query, return_tensors="pt", truncation=True, max_length=max_length)

        # Generate the output
        outputs = model.generate(**encoding)

        # Decode the output and append to predictions
        prediction = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.append(prediction)

    return predictions


queries = []
print("Please enter your queries one by one. When you're finished, simply press Enter.")

while True:
    query = input("Enter query (leave blank to finish): ")
    if not query.strip():
        break
    queries.append(query)


# Generate and decode predictions for each query
predictions = predict_with_model(model, tokenizer, data, queries)

# Print predictions
for query, prediction in zip(queries, predictions):
    print(f"Query: {query}")
    print(f"Prediction: {prediction}")
    print()

Please enter your queries one by one. When you're finished, simply press Enter.
Enter query (leave blank to finish): which sales type has more sold
Enter query (leave blank to finish): 
Query: which sales type has more sold
Prediction: [' office supplies']

